# Generate config for sensor-mqtt.yaml and sensor.yaml

This notebook generates the required config for `sensor-mqtt.yaml` and `sensor.yaml` housed at https://github.com/amadeuspzs/homeassistant-config

It requires a `sensors.json` file with topics and names of sensors.

In [ ]:
import json
import yaml
from slugify import slugify

```yaml
mqtt:
    sensor:
    - state_topic: "drawingroom/temperature"
      name: "Drawing Room Temperature"
      unit_of_measurement: "°C"
      device_class: "temperature"
    - state_topic: "drawingroom/humidity"
      name: "Drawing Room Humidity"
      unit_of_measurement: "%"
      device_class: "humidity"
    - state_topic: "drawingroom/vbatt"
      name: "Drawing Room Battery"
      unit_of_measurement: "V"
      device_class: "voltage"
```  

```yaml
- platform: template
  sensors:
    drawing_room_power:
      friendly_name: "Drawing Room Power"
      unit_of_measurement: "%"
      value_template: "{{ (100*(states('sensor.living_room_k_a_battery') | float) /4.2) | round(1) }}"
      device_class: battery
```

In [ ]:
with open("sensors.json") as f:
     sensors = json.load(f)

mqtt_yaml_sensors = ""
yaml_sensors = ""
for sensor in sensors:
    entity_id = slugify(sensor["name"], separator="_")
    mqtt_yaml_sensors += f'''
- state_topic: "{sensor["topic"]}/temperature"
  name: "{sensor["name"]} Temperature"
  unit_of_measurement: "°C"
  device_class: "temperature"
- state_topic: "{sensor["topic"]}/humidity"
  name: "{sensor["name"]} Humidity"
  unit_of_measurement: "%"
  device_class: "humidity"
- state_topic: "{sensor["topic"]}/vbatt"
  name: "{sensor["name"]} Battery"
  unit_of_measurement: "V"
  device_class: "voltage"
'''
    yaml_sensors += f'''
- platform: template
  sensors:
    {entity_id}_power:
      friendly_name: "{sensor["name"]} Power"
      unit_of_measurement: "%"
      value_template: "{{% set power = (100*(states('sensor.{entity_id}_battery') | float)/4.2) | round(1) %}} {{{{ power if power <= 100 else 100 }}}}"
      device_class: battery
'''
    

try:
    yaml.safe_load(mqtt_yaml_sensors)
    yaml.safe_load(yaml_sensors)
    print(mqtt_yaml_sensors)
    print(yaml_sensors)
except:
    print("YAML failed to parse")